In [70]:
import pandas as pd

train_data = pd.read_csv("data/train.csv")
train_data = train_data.drop(['Name', 'Cabin', 'Ticket'], axis = 1)
test_data = pd.read_csv("data/test.csv")
test_data = test_data.drop(['Name', 'Cabin', 'Ticket'], axis = 1)

con = train_data['Sex'] == 'female'
con2 = train_data['Sex'] == 'male'

train_data.loc[con, 'Age'] = train_data.loc[con, 'Age'].fillna(train_data.loc[con, 'Age'].mean())
train_data.loc[con2, 'Age'] = train_data.loc[con2, 'Age'].fillna(train_data.loc[con2, 'Age'].mean())
train_data['Embarked'] = train_data['Embarked'].fillna('S')

test_data.loc[con, 'Age'] = test_data.loc[con, 'Age'].fillna(test_data.loc[con, 'Age'].mean())
test_data.loc[con2, 'Age'] = test_data.loc[con2, 'Age'].fillna(test_data.loc[con2, 'Age'].mean())
test_data['Embarked'] = test_data['Embarked'].fillna('S')
test_data['Fare'] = test_data['Fare'].fillna(test_data['Fare'].mean())

train_data

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.000000,1,0,7.2500,S
1,2,1,1,female,38.000000,1,0,71.2833,C
2,3,1,3,female,26.000000,0,0,7.9250,S
3,4,1,1,female,35.000000,1,0,53.1000,S
4,5,0,3,male,35.000000,0,0,8.0500,S
...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,27.000000,0,0,13.0000,S
887,888,1,1,female,19.000000,0,0,30.0000,S
888,889,0,3,female,27.915709,1,2,23.4500,S
889,890,1,1,male,26.000000,0,0,30.0000,C


In [71]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate

train_target = train_data["Survived"]
features = ['Age', 'Fare', 'Pclass', 'Sex', 'Parch', 'SibSp']

train_input = pd.get_dummies(train_data[features])
test_input = pd.get_dummies(test_data[features])
print(train_input)

           Age     Fare  Pclass  Parch  SibSp  Sex_female  Sex_male
0    22.000000   7.2500       3      0      1       False      True
1    38.000000  71.2833       1      0      1        True     False
2    26.000000   7.9250       3      0      0        True     False
3    35.000000  53.1000       1      0      1        True     False
4    35.000000   8.0500       3      0      0       False      True
..         ...      ...     ...    ...    ...         ...       ...
886  27.000000  13.0000       2      0      0       False      True
887  19.000000  30.0000       1      0      0        True     False
888  27.915709  23.4500       3      2      1        True     False
889  26.000000  30.0000       1      0      0       False      True
890  32.000000   7.7500       3      0      0       False      True

[891 rows x 7 columns]


In [72]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures()
poly.fit(train_input)
train_poly = poly.transform(train_input)
poly.fit(test_input)
test_poly = poly.transform(test_input)
# print(train_poly.shape)

In [73]:
model.fit(train_poly, train_target)

RandomForestClassifier(max_depth=10, min_samples_split=5, oob_score=True,
                       random_state=42)

In [74]:
print(model.score(train_poly, train_target))

0.9214365881032548


In [75]:
model = RandomForestClassifier(oob_score=True, max_depth= 10, max_features= 'sqrt', min_samples_leaf= 1, min_samples_split= 5, n_estimators= 100, random_state= 42)
model.fit(train_poly, train_target)
predictions = model.predict(train_poly)
scores = cross_validate(model, train_poly, train_target, return_train_score = True, n_jobs = 1)
print(scores)
print(model.oob_score_)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

{'fit_time': array([0.23232532, 0.22330165, 0.2374413 , 0.2368567 , 0.24104166]), 'score_time': array([0.00844288, 0.00813317, 0.009799  , 0.00875354, 0.00834322]), 'test_score': array([0.79329609, 0.8258427 , 0.87078652, 0.82022472, 0.84831461]), 'train_score': array([0.92696629, 0.93828892, 0.9200561 , 0.92706872, 0.914446  ])}
0.8294051627384961


ValueError: array length 891 does not match index length 418